# A - Pay to Win
### A枚のコインを支払い、持っている数を 2倍する。
### B枚のコインを支払い、持っている数を 3倍する。
### C枚のコインを支払い、持っている数を 5倍する。
### D枚のコインを支払い、持っている数を 1増やす、または 1減らす。

In [1]:
import sys
sys.setrecursionlimit(10**8)   # 再帰回数の制限！！
T = int(input())
for _ in range(T):
    N,A,B,C,D = map(int,input().split())
    INF = float('inf')
    mem = {}    # 過去に計算した際の結果を保存、無意味に計算を増やさない工夫
    def solve(n):
        if n==0: return 0
        if n==1: return D
        if n in mem: return mem[n]    # 過去の結果を使って出力
        
        # A<B<C<Dの順なので、その順番で場合分け
        # 割り切れる場合、割り切れるまで減らす場合、割り切れるまで増やす場合
        tmp = n*D
        if n%2==0:
            tmp = min(tmp, solve(n//2) + A)
        else:
            tmp = min(tmp, solve(n//2) + A+D, solve(n//2+1) + A+D)
        if n%3==0:
            tmp = min(tmp, solve(n//3) + B)
        else:
            d = n%3
            u = 3-d
            tmp = min(tmp, solve(n//3) + B+d*D, solve(n//3+1) + B+u*D)
        if n%5==0:
            tmp = min(tmp, solve(n//5) + C)
        else:
            d = n%5
            u = 5-d
            tmp = min(tmp, solve(n//5) + C+d*D, solve(n//5+1) + C+u*D)
        mem[n] = tmp
        return tmp
    print(solve(N))

5
11 1 2 4 8
20
11 1 2 2 8
19
32 10 8 5 4
26
29384293847243 454353412 332423423 934923490 1
3821859835
900000000000000000 332423423 454353412 934923490 987654321
23441258666


In [ ]:
# 時間かかり過ぎ
def dist(n):
    if n == 0:
        return 0
    if n == 1:
        return D
    
    ret = min(
        D * n,
        D * abs(n - n//5*5) + C + dist(n//5),
        D * abs(n - (n+4)//5*5) + C + dist((n+4)//5),
        D * abs(n - n//3*3) + B + dist(n//3),
        D * abs(n - (n+2)//3*3) + B + dist((n+2)//3),
        D * abs(n - n//2*2) + A + dist(n//2),
        D * abs(n - (n+1)//2*2) + A + dist((n+1)//2)
    )

    return ret

# import sys
# sys.setrecursionlimit(10**8)
T = int(input())
for _ in range(T):
    N,A,B,C,D = map(int,input().split())
    print(dist(N))
    

5
29384293847243 454353412 332423423 934923490 1


# B - Joker

In [1]:
# 並列処理numbaの例
import random
from numba import jit, prange
import time


@jit(nopython=True, parallel=True)
def calc_pi(NUM):
    counter = 0
    for i in prange(NUM):
        x = random.random()
        y = random.random()
        if x*x+y*y < 1.0:
            counter += 1
    pi = 4.0*counter/NUM
    return pi


def main():
    NUM = 1000000000
    start = time.time()
    pi = calc_pi(NUM)
    end = time.time()
    print("pi, {}, elapsed={}".format(pi, end-start))

if __name__ == '__main__':
    main()

pi, 3.141591548, elapsed=3.9707651138305664


In [2]:
# 並列処理なし
def calc_pi(NUM):
    counter = 0
    for i in range(NUM):
        x = random.random()
        y = random.random()
        if x*x+y*y < 1.0:
            counter += 1
    pi = 4.0*counter/NUM
    return pi


def main():
    NUM = 1000000000
    start = time.time()
    pi = calc_pi(NUM)
    end = time.time()
    print("pi, {}, elapsed={}".format(pi, end-start))

if __name__ == '__main__':
    main()

pi, 3.141545936, elapsed=251.83397674560547


In [ ]:
from numba import njit
import numpy as np

def cc_export():
    @njit('(i8, i8, i8[:], i8[:], i8[:], i8)')
    def update(n, w, que, there, cost, pos):
        que[0] = pos
        i, l = 0, 1
        while i < l:
            pos = que[i]
            i += 1
            c = cost[pos]
            t = there[pos]
            k = c + t
            for df in (-w, w, -1, 1):
                pp = pos + df
                if cost[pp] > k:
                    cost[pp] = k
                    que[l] = pp
                    l += 1

    @njit('(i8, i8[:])')
    def main(n, p):
        w = n + 2
        que = np.zeros(4 * w * w, dtype=np.int64)
        there = np.ones(w * w, dtype=np.int64)
        # there = np.array([
        #     1 if 1 <= i and i <= n and 1 <= j and j <= n else 0
        #     for j in range(w) for i in range(w)
        # ],
        #                  dtype=np.int64)
        cost = np.array([
            -1 if i == 0 or i == n + 1 or j == 0 or j == n + 1 else min(
                i - 1, n - i, j - 1, n - j) for j in range(w) for i in range(w)
        ],
                        dtype=np.int64)

        ans = 0
        for pos in p:
            # v = list(map(lambda x: ' ' if x == 0 else '*', list(there)))
            # for i in range(w):
            #     print(' '.join(v[i * w:(i + 1) * w]))
            # print(cost.reshape((w, w)))
            # print("add", cost[pos], "for", divmod(pos, w))
            ans += cost[pos]
            there[pos] = 0
            update(n, w, que, there, cost, pos)
        return ans

    from numba.pycc import CC
    cc = CC('solution')
    cc.export('update', '(i8, i8, i8[:], i8[:], i8[:], i8)')(update)
    cc.export('main', '(i8, i8[:])')(main)
    cc.compile()


if __name__ == '__main__':
    import sys
    if sys.argv[-1] == 'ONLINE_JUDGE':
        cc_export()
        sys.exit(0)

    def transform(t):
        y, x = divmod(int(t) - 1, n)
        return (y + 1) * (n + 2) + (x + 1)

    from solution import update, main
    n = int(input())
    p = np.array(list(map(transform, input().split())), dtype=np.int64)
    print(main(n, p))